In [66]:
import pandas as pd
import numpy as np
from scipy.optimize import linear_sum_assignment
import random
import math
import time

In [67]:
# Lade die CSV-Dateien mit Personen, Teilaufgaben und Projekten
personen_df = pd.read_csv("../data/personen.csv")
teilaufgaben_df = pd.read_csv("../data/teilaufgaben.csv")

In [68]:
# Wandle die Kompetenzen in eine Liste um, um Filterung zu erleichtern
personen_df["kompetenzen_liste"] = personen_df["kompetenzen"].str.split(r",\s*")

print(f"Loaded: {len(personen_df)} people, {len(teilaufgaben_df)} tasks")
print(f"Total workload: {teilaufgaben_df['aufwand'].sum():.1f} hours")

Loaded: 150 people, 8 tasks
Total workload: 19.5 hours


In [61]:
# Belegungstracker: Speichert die bereits verplante Kapazität je Person und Monat
belegungen = {}

In [62]:
# Liste zur Speicherung der Zuweisungsergebnisse
matching_ergebnisse = []

In [69]:
def berechne_verfuegbarkeit(person, ta_start, ta_ende, belegungen, ta_projekt_id):
    """
    Improved availability calculation with project assignment check
    """
    person_id = person["id"]
    verfügbar = True
    verfügbarkeit_summe = 0
    monatliche_restwerte = {}

    for monat in range(ta_start, ta_ende + 1):
        monat_label = f"{monat:02d}/2025"
        
        # Check existing project assignment
        belegung_label = f"projektbelegung_{monat_label}"
        aktuelle_belegung = person.get(belegung_label, "Frei")
        
        if aktuelle_belegung not in ["Frei", ta_projekt_id]:
            rest = 0  # Person already assigned to other project
        else:
            verfügbarkeit = person.get(f"verfuegbarkeit_{monat_label}", 0)
            belegt = belegungen.get(person_id, {}).get(monat_label, 0)
            rest = max(0, verfügbarkeit - belegt)

        monatliche_restwerte[monat_label] = rest

        if rest <= 0:
            verfügbar = False
            break
        verfügbarkeit_summe += rest

    return verfügbar, verfügbarkeit_summe, monatliche_restwerte

def calculate_person_score(person, ta, belegungen=None):
    """
    Calculate person score for a task
    """
    if belegungen is None:
        belegungen = {}
        
    ta_start = int(ta["start"].split("/")[0])
    ta_ende = int(ta["ende"].split("/")[0])
    
    verfügbar, verf_summe, _ = berechne_verfuegbarkeit(person, ta_start, ta_ende, belegungen, ta["projekt_id"])
    
    if verfügbar and verf_summe > 0:
        # Score based on availability, time budget and skill match
        kompetenz_bonus = 1.0
        if ta["kompetenz"] in person["kompetenzen_liste"]:
            kompetenz_bonus = 1.5  # Bonus for exact skill match
            
        score = verf_summe * person["zeitbudget"] * kompetenz_bonus
        return round(score, 2)
    
    return 0

In [64]:
def backtrack_matching(teilaufgaben_df, personen_df, belegungen, current_ta_index=0, matching_ergebnisse=None):
    """
    Backtracking-Funktion zum Finden alternativer Zuweisungen, wenn der Greedy-Ansatz fehlschlägt.
    
    Args:
        teilaufgaben_df: DataFrame mit Teilaufgaben
        personen_df: DataFrame mit Personen
        belegungen: Aktuelle Belegungen
        current_ta_index: Aktueller Index der Teilaufgabe
        matching_ergebnisse: Aktuelle Matching-Ergebnisse
        
    Returns:
        tuple: (bool erfolgreich, list matching_ergebnisse)
    """
    if matching_ergebnisse is None:
        matching_ergebnisse = []
        
    # Basisfall: Alle Teilaufgaben zugewiesen
    if current_ta_index >= len(teilaufgaben_df):
        return True, matching_ergebnisse
        
    ta = teilaufgaben_df.iloc[current_ta_index]
    ta_start = int(ta["start"].split("/")[0])
    ta_ende = int(ta["ende"].split("/")[0])
    ta_monate = ta_ende - ta_start + 1
    ta_monatsaufwand = ta["aufwand"] / ta_monate
    restaufwand = ta["aufwand"]
    
    # Alle möglichen Kandidaten für diese Teilaufgabe finden
    kandidaten = []
    for _, person in personen_df.iterrows():
        if ta["kompetenz"] not in person["kompetenzen_liste"]:
            continue
            
        verfügbar, verf_summe, monatliche_restwerte = berechne_verfuegbarkeit(person, ta_start, ta_ende, belegungen)
        
        if verfügbar and verf_summe > 0:
            score = round(verf_summe * person["zeitbudget"], 2)
            kandidaten.append((person, score, verf_summe, monatliche_restwerte))
    
    # Kandidaten nach Score sortieren
    kandidaten = sorted(kandidaten, key=lambda x: x[1], reverse=True)
    
    # Jeden Kandidaten ausprobieren
    for person, score, verf_summe, eintrag_monate in kandidaten:
        # Kopie des aktuellen Zustands erstellen
        temp_belegungen = {k: v.copy() for k, v in belegungen.items()}
        temp_matching = matching_ergebnisse.copy()
        
        pid = person["id"]
        if pid not in temp_belegungen:
            temp_belegungen[pid] = {}
            
        verfügbarkeit_gesamt = sum(eintrag_monate.values())
        if verfügbarkeit_gesamt == 0:
            continue
            
        anteil = min(restaufwand, verfügbarkeit_gesamt)
        monatsanteil = anteil / ta_monate
        
        # Versuchen, diese Person zuzuweisen
        temp_matching.append({
            "teilaufgabe_id": ta["teilaufgabe_id"],
            "teilaufgabe": ta["bezeichnung"],
            "projekt_id": ta["projekt_id"],
            "kompetenz": ta["kompetenz"],
            "person_id": pid,
            "name": person["name"],
            "zugewiesener_aufwand": round(anteil, 2)
        })
        
        # Belegungen aktualisieren
        for monat_label in eintrag_monate:
            verbrauch = min(monatsanteil, eintrag_monate[monat_label])
            temp_belegungen[pid][monat_label] = temp_belegungen[pid].get(monat_label, 0) + verbrauch
            
        # Rekursiv versuchen, die restlichen Teilaufgaben zuzuweisen
        success, results = backtrack_matching(
            teilaufgaben_df, 
            personen_df, 
            temp_belegungen, 
            current_ta_index + 1, 
            temp_matching
        )
        
        if success:
            return True, results
            
    return False, matching_ergebnisse

# Hauptloop über alle Teilaufgaben
for _, ta in teilaufgaben_df.iterrows():
    ta_start = int(ta["start"].split("/")[0])
    ta_ende = int(ta["ende"].split("/")[0])
    ta_monate = ta_ende - ta_start + 1
    ta_monatsaufwand = ta["aufwand"] / ta_monate
    restaufwand = ta["aufwand"]
    kandidaten = []

    # Filtere Personen, die die benötigte Kompetenz besitzen
    for _, person in personen_df.iterrows():
        if ta["kompetenz"] not in person["kompetenzen_liste"]:
            continue

        verfügbar, verf_summe, monatliche_restwerte = berechne_verfuegbarkeit(person, ta_start, ta_ende, belegungen)

        if verfügbar and verf_summe > 0:
            score = round(verf_summe * person["zeitbudget"], 2)
            kandidaten.append((person, score, verf_summe, monatliche_restwerte))

    # Sortiere Kandidaten nach ihrer nutzbaren Kapazität (absteigend)
    kandidaten = sorted(kandidaten, key=lambda x: x[1], reverse=True)

    # Weise Kapazität zu, bis Aufwand gedeckt ist oder keine Ressourcen mehr vorhanden
    for person, score, verf_summe, eintrag_monate in kandidaten:
        if restaufwand <= 0:
            break

        pid = person["id"]
        if pid not in belegungen:
            belegungen[pid] = {}

        verfügbarkeit_gesamt = sum(eintrag_monate.values())
        if verfügbarkeit_gesamt == 0:
            continue

        anteil = min(restaufwand, verfügbarkeit_gesamt)
        monatsanteil = anteil / ta_monate

        # Speichere Matching-Ergebnis
        matching_ergebnisse.append({
            "teilaufgabe_id": ta["teilaufgabe_id"],
            "teilaufgabe": ta["bezeichnung"],
            "projekt_id": ta["projekt_id"],
            "kompetenz": ta["kompetenz"],
            "person_id": pid,
            "name": person["name"],
            "zugewiesener_aufwand": round(anteil, 2)
        })

        # Aktualisiere belegte Verfügbarkeiten
        for monat_label in eintrag_monate:
            verbrauch = min(monatsanteil, eintrag_monate[monat_label])
            belegungen[pid][monat_label] = belegungen[pid].get(monat_label, 0) + verbrauch

        restaufwand -= anteil

    print(f"Verfügbare Kandidaten für: {ta['bezeichnung']} → {len(kandidaten)}")

# Prüfen, ob alle Teilaufgaben zugewiesen wurden
total_assigned = sum(item["zugewiesener_aufwand"] for item in matching_ergebnisse)
total_needed = teilaufgaben_df["aufwand"].sum()

if total_assigned < total_needed:
    print("Greedy-Ansatz konnte keine vollständige Lösung finden. Versuche Backtracking...")
    success, matching_ergebnisse = backtrack_matching(teilaufgaben_df, personen_df, {})
    if success:
        print("Backtracking hat eine vollständige Lösung gefunden!")
    else:
        print("Warnung: Auch mit Backtracking konnte keine vollständige Lösung gefunden werden")



Verfügbare Kandidaten für: Entwurf CAD-Baugruppe → 0
Verfügbare Kandidaten für: Simulation FEM-Struktur → 0
Verfügbare Kandidaten für: Signalverarbeitung Sensor → 0
Verfügbare Kandidaten für: Entwicklung Python-Modul → 0
Verfügbare Kandidaten für: Statistische Auswertung → 0
Verfügbare Kandidaten für: Optische Analyse → 0
Verfügbare Kandidaten für: Datenbankmodellierung → 0
Verfügbare Kandidaten für: Thermodynamische Bewertung → 0
Greedy-Ansatz konnte keine vollständige Lösung finden. Versuche Backtracking...
Warnung: Auch mit Backtracking konnte keine vollständige Lösung gefunden werden


In [65]:
df_matching = pd.DataFrame(matching_ergebnisse)
df_matching.to_csv("matching_ergebnis.csv", index=False)
print("Matching abgeschlossen! Ergebnisse gespeichert in 'matching_ergebnis.csv'")

Matching abgeschlossen! Ergebnisse gespeichert in 'matching_ergebnis.csv'
